In [226]:
import numpy as np
from collections import Counter, OrderedDict
import collections
import pandas as pd

In [227]:
def readFile(filePath): 
    with open (filePath, 'r') as f: 
        lines = f.readlines()
        text = []
        label = []
        
        temp_arr = []
        for l in lines: 
            if l != "\n":
                l_split = l.strip().split(" ")
                te = " ".join(l_split[:-1])
                la = l_split[-1]
                temp_arr.append((te, la))
                label.append(l_split[-1])
            else: 
                text.append(temp_arr)
                temp_arr = []
    return text, label

In [228]:
def transition_table(text, tra_arr):
    for ct, t in enumerate(text): 
        for ci, i in enumerate(t):
            if ci == len(t)-1:
                tra_psn = list(final_dict.keys()).index(i[1])
                tra_arr[tra_psn][len(final_dict)-1] += 1/final_dict[i[1]]
            else:
                if ci == 0: 
                    tra_psn = list(final_dict.keys()).index(i[1])
                    tra_arr[0][tra_psn-1] += 1/final_dict['START']
                cur_psn = list(final_dict.keys()).index(i[1])  
                next_psn = list(final_dict.keys()).index(t[ci+1][1])
                tra_arr[cur_psn][next_psn-1] += 1/final_dict[i[1]]
    return tra_arr
    

In [229]:
def vertibi_algo(tra_arr, em_arr, final_dict, n): 
    # forward 
    rst = []
    pi_arr = []
    dy_arr = np.ones(em_arr.shape[0]) #[1,1,1]
    for c, i in enumerate (n):
        if c == 0: 
            temp_arr = []
            for j in range(em_arr.shape[0]):
                val = dy_arr[0] * tra_arr [0][j] * em_arr[j][i]
                temp_arr.append(val)
            pi_arr.append(temp_arr)
            dy_arr = temp_arr
        else: 
            temp_arr = [] #1/216, 1/128, 1/72
            for a in range(em_arr.shape[0]): # X 
                pi_temp = []
                for b in range(em_arr.shape[0]): # XYZ in X 
                    pi_temp.append(dy_arr[b]* tra_arr[b+1][a] * em_arr[a][i])
                temp_arr.append(max(pi_temp))
            pi_arr.append(temp_arr)
            dy_arr = temp_arr
    #yn to stop
    temp_arr = []
    for f in range(em_arr.shape[0]):
        temp_arr.append(dy_arr[f] * tra_arr[f+1][tra_arr.shape[1]-1])
    rst.append(list(final_dict.keys())[np.argmax(temp_arr)+1])    
        
    idx_last = list(final_dict.keys()).index(rst[0])-1
    for i in reversed(range(len(n)-1)):
        temp_arr = []
        for j in range(em_arr.shape[0]):
            temp_arr.append(pi_arr[i][j] * tra_arr[j+1][idx_last])
        idx_last = np.argmax(temp_arr)
        rst.append(list(final_dict.keys())[idx_last+1])
        
    return list(reversed(rst))

In [255]:
def preprocess(filepath, lan):
    
    em_arr = pd.read_csv(filepath, sep=',',header=None)
#     em_arr = em_arr.T
    em_arr = em_arr.drop([0, 2])
    em_arr.columns = em_arr.iloc[0]
    em_arr = em_arr[1:]
    if lan == 'EN':
        em_arr = em_arr.reindex([5,6,11,12,4,7,9,3,16,10,8,19,20,18,21,17,13,22,23,14,15])
    elif lan == 'CN':
        em_arr = em_arr.reindex([4,6,3,7,8,5,9])
    else: 
         em_arr = em_arr.reindex([6,3,4,7,5,9,8])
    header_list = list(em_arr.columns.values)
    em = em_arr.drop(['Text'], axis=1).values
    em = em.astype('float64')
    header_list = header_list[1:]
    return em, header_list

In [250]:
def get_label_index(filepath, header_list):
    test = []
    output_index = []
    output_label = []
    index = []
    label = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
        for l in lines: 
            if l != "\n":
                label.append(l.strip())
                try: 
                    index.append(header_list.index(l.strip()))
                except: 
                    index.append(header_list.index("#UNK#"))
            else: 
                output_index.append(index)
                output_label.append(label)
                index = []
                label = []
    return output_index, output_label



In [251]:
def get_output(tra_arr, em, final_dict, index, label):
    seq = []
    for ci, i in enumerate(index): 
        seq_temp = vertibi_algo(tra_arr, em, final_dict, i)
        temp_arr = []
        for cj, j in enumerate(seq_temp): 
            temp_arr.append("{} {}".format(label[ci][cj],j))
        seq.append(temp_arr)

    with open ('dev.p3.out', 'w') as f: 
        for i in seq: 
            for j in i: 
                f.write(j+"\n")
            f.write("\n")
    return "Done"

In [252]:
def get_label_dict(text, label): 
    final_dict = {'START': len(text)}
    label_counter = dict(Counter(label))
    final_dict.update(label_counter)
    return final_dict 

In [256]:
def get_tra_array(final_dict):
    tra_arr = np.zeros((len(final_dict),len(final_dict)))
    tra_arr = transition_table(text, tra_arr)
    return tra_arr

In [257]:
pre_filepath = '/Users/geraldlim/Desktop/CN/CN.csv'
devin_filepath = '/Users/geraldlim/Desktop/CN/dev.in'
train_filepath = '/Users/geraldlim/Desktop/CN/train'
text, label = readFile(train_filepath)
final_dict = get_label_dict(text, label)
tra_arr = get_tra_array(final_dict)
em, header_list = preprocess(pre_filepath, 'CN') #CSV
index, output_label = get_label_index(devin_filepath, header_list) #dev in
print(get_output(tra_arr, em, final_dict, index, output_label))

Done
